In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox

## Parameters for the loading programs to statbank

date	
> The date the table should be published to Statistikkbanken

In [ ]:
date = widgets.DatePicker(
    description='Pick date',
    disabled=False
)

path	
> Path for the loadingfiles placement. Should end with a slash (/)

In [ ]:
path = widgets.Text(
    value='/ssb/stamme01/',
    placeholder='Path',
    description='Path:',
    disabled=False
)

file	
> Name of the datafile to be loaded. Case-sensitive.

In [ ]:
file = widgets.Text(
    value='bmekurstab41.dat',
    placeholder='File name',
    description='File:',
    disabled=False
)

tablename	
> Name of table to load

In [ ]:
tablename = widgets.Text(
    value='bmekurstab4',
    placeholder='Tablename',
    description='Tablename:',
    disabled=False
)

overwrite	
> Automatically overwrite duplicates on time

In [ ]:
overwrite = widgets.Checkbox(
    value=True,
    description='Overwrite duplicates on time',
    disabled=False,
    indent=False
)

approval	
> How and when should the data be approved?
>>0 = Manual approval \
1 = Automatic after load (immediatly)\
2 = JIT (Just In Time, right before publishing time)

In [ ]:
approval = widgets.Dropdown(
    options=['Manual approval', 'Immediatly', 'Just In Time'],
    value='Manual approval',
    description='Approval:',
    disabled=False,
)

dbase	
> Server to load onto. TEST, QA or PROD

In [ ]:
dbase = widgets.Dropdown(
    options=['TEST', 'QA', 'PROD'],
    value='TEST',
    description='Server:',
    disabled=False,
)

user
> Oracle userid for the person loading the data.

In [ ]:
user = widgets.Text(
    value='brukernavn',
    placeholder='Oracle user id',
    description='OracleUser:',
    disabled=False
)

password	
> Oracle-password for the userid above.

In [ ]:
password = widgets.Password(
    value='passord',
    placeholder='Oracle password',
    description='OraclePass:',
    disabled=False
)


initials
> User to recieve email about the load. 

In [ ]:
initials = widgets.Text(
    value='aaa',
    placeholder='initials',
    description='Mail-initials:',
    disabled=False
)

## Fill Parameters

In [ ]:
VBox([
    HBox([user, password, initials]),
    HBox([date, overwrite]),
    HBox([dbase, approval, ]),
    HBox([path, file, tablename])
    
])

## Summary Parameters

In [ ]:
print(
    'Date:\t\t', date.value, '\n',
    'Path:\t', path.value, '\n',
    'File:\t\t', file.value, '\n',
    'Tabele:\t', tablename.value, '\n',
    'Overwrite:\t', overwrite.value, '\n',
    'Approval:\t', approval.value, '\n',
    'Server:\t', dbase.value, '\n',
    'User:\t', user.value, '\n',
    'Password:\t', len(password.value) * '*', '\n',
    'Email:\t\t', f'{initials.value}@ssb.no'
)

### Convert parameters
Also recoding to match what the perl and bash-scripts expect.

In [ ]:
user = user.value
password = password.value
path = path.value
file = file.value
tablename = tablename.value
dato = date.value
initials = initials.value
overskriv = overwrite.value
godkjenn = approval.value

## Run external scripts

In [ ]:
perl_path = f'/ssb/stamme04/statbas/system/prog/filoverforing/{dbase.value}/overforing.pl'
perl_path

In [ ]:
laste_path = f'/ssb/stamme04/system/prog/bestillLasting/{dbase.value}/bestillLasting.sh'
laste_path

In [ ]:
# Remove old log
!rm /tmp/$user\_overforing_sas.log

In [ ]:
# Run perl script
!$perl_path -o /tmp -a $user $password $path$file > /tmp/$user\_overforing_sas.log 2>&1

In [ ]:
# Terminal command
#f'{perl_path} -o /tmp -a {user} {password} {path}{file} > /tmp/{user}_overforing_sas.log 2>&1'

In [ ]:
# List files in /tmp/
#import os
#for filename in os.listdir('/tmp/'): print(filename)

In [ ]:
# Read the log
log = open(f'/tmp/{user}_overforing_sas.log')
# Print the log
print(log.read())

In [ ]:
# Only if the log contains a message of success do we continue
if 'Programmet er ferdig' in log.read():
    # Remove the order-log
    !rm /tmp/$user\_bestillLasting_sas.log
    # Run bash-script to order the load
    !$laste_path $user $password $tablename $dato 10:00 $initials $overskriv $godkjenn > /tmp/$user\_bestillLasting_sas.log 2>&1
    # Alternative truncated version of call, will set defaults on last 4 parameters: "10:00", "whoami" = initials, overwrite: "1", approval: "2"
    #!$laste_path $user $password $tablename $dato > /tmp/$user\_bestillLasting_sas.log 2>&1
    # Open the order-log
    laste_log = open(f'/tmp/{user}_bestillLasting_sas.log')
    # Print order-log
    print(laste_log.read())

In [ ]:
# Delete password from memory
del password